Datasets include:

```python
dark_slick_vessel_source.csv
dark_slick_infra_source.csv
infra_slick_vessel_source.csv
infra_slick_dark_source.csv
vessel_slick_infra_source.csv
vessel_slick_dark_source.csv
```

Columns - slick_id, source_type, slick_type, coincidence_score, truth

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from collation_optimizer import CollationOptimizer
import pandas as pd

In [ ]:
import os
import sys
from dotenv import load_dotenv

load_dotenv()

# Set the download path for demonstration and ensure the folder exists.
download_path = os.getenv("ASA_DOWNLOAD_PATH")
os.makedirs(download_path, exist_ok=True)

git_path = os.getenv("GIT_FOLDER")
cv3_path = os.getenv("CV3_FOLDER")
sys.path.append(git_path)
sys.path.append(cv3_path)

In [ ]:
collation_datasets_folder = (
    cv3_path + "/asa_analysis/evaluation/saved_results/collation datasets/"
)
date = "3-25-2025c"
dark_slick_vessel_source = pd.read_csv(
    f"{collation_datasets_folder}/{date}/dark_slick_vessel_source.csv"
)
dark_slick_infra_source = pd.read_csv(
    f"{collation_datasets_folder}/{date}/dark_slick_infra_source.csv"
).drop(columns=["Unnamed: 0"])
infra_slick_vessel_source = pd.read_csv(
    f"{collation_datasets_folder}/{date}/infra_slick_vessel_source.csv"
).drop(columns=["Unnamed: 0"])
infra_slick_dark_source = pd.read_csv(
    f"{collation_datasets_folder}/{date}/infra_slick_dark_source.csv"
).drop(columns=["Unnamed: 0"])
vessel_slick_dark_source = pd.read_csv(
    f"{collation_datasets_folder}/{date}/vessel_slick_dark_source.csv"
)
vessel_slick_infra_source = pd.read_csv(
    f"{collation_datasets_folder}/{date}/vessel_slick_infra_source.csv"
).drop(columns=["Unnamed: 0"])
dark_mean = dark_slick_vessel_source[dark_slick_vessel_source["truth"]][
    "coincidence_score"
].mean()
dark_std = dark_slick_vessel_source[dark_slick_vessel_source["truth"]][
    "coincidence_score"
].std()
vessel_mean = vessel_slick_dark_source[vessel_slick_dark_source["truth"]][
    "coincidence_score"
].mean()
vessel_std = vessel_slick_dark_source[vessel_slick_dark_source["truth"]][
    "coincidence_score"
].std()
infra_mean = infra_slick_vessel_source[infra_slick_vessel_source["truth"]][
    "coincidence_score"
].mean()
infra_std = infra_slick_vessel_source[infra_slick_vessel_source["truth"]][
    "coincidence_score"
].std()


print("default means")
print("vessel_mean:", vessel_mean)
print("vessel_std:", vessel_std)
print("infra_mean:", infra_mean)
print("infra_std:", infra_std)
print("dark_mean:", dark_mean)
print("dark_std:", dark_std)

In [ ]:
slick_source_dict = {
    "dark_slick_infra_source": dark_slick_infra_source,
    "dark_slick_vessel_source": dark_slick_vessel_source,
    "infra_slick_vessel_source": infra_slick_vessel_source,
    "infra_slick_dark_source": infra_slick_dark_source,
    "vessel_slick_infra_source": vessel_slick_infra_source,
    "vessel_slick_dark_source": vessel_slick_dark_source,
}

In [ ]:
# incorporate mean and std constants from above into a dictionary that takes in 1 for vess, 2 for infra, and 3 for dark and returns mean and std tuple
# mean_std_dict = {1: (vessel_mean,vessel_std), 2: (infra_mean,infra_std), 3: (dark_mean,dark_std)}
mean_std_dict = {1: (0, 1), 2: (0, 1), 3: (0, 1)}
iter_params = []
collation_optimizer = CollationOptimizer(
    mean_std_dict, slick_source_dict, iter_params, ignore_lone_truth=True
)
collation_optimizer.compute_collation_performance()
collation_optimizer.targets

In [ ]:
mean_std_dict = {1: (vessel_mean, vessel_std), 2: (0, 1), 3: (0, 1)}
collation_optimizer.mean_std_dict = mean_std_dict
collation_optimizer.compute_collation_performance()
collation_optimizer.targets

In [ ]:
params_dark = {
    "fix_type": 2,
    "optimize_type": 3,
    "mean_range": [0, 1],
    "std_range": [0, 1],
    "search_count": 40,
}
params_infra = {
    "fix_type": 3,
    "optimize_type": 2,
    "mean_range": [0, 1],
    "std_range": [0, 1],
    "search_count": 40,
}
iter_params = [params_infra, params_dark] * 3

In [ ]:
mean_std_dict = {1: (vessel_mean, vessel_std), 2: (0, 1), 3: (0, 1)}
collation_optimizer = CollationOptimizer(
    mean_std_dict, slick_source_dict, iter_params, ignore_lone_truth=True
)
collation_optimizer.perform_infra_dark_gradient_ascent()

In [ ]:
collation_optimizer.plot_target_over_iterations()

In [ ]:
collation_optimizer.mean_std_dict

In [ ]:
collation_optimizer.compute_collation_performance()
collation_optimizer.targets

In [ ]:
collation_optimizer.ignore_lone_truth = False
collation_optimizer.compute_collation_performance()
collation_optimizer.targets